# Tasks

For this assignment, you will be working with the Yelp API.

As before, you will use the Yelp API to search your favorite city for a cuisine type of your choice.

Extract all of the results from your search and compile them into one dataframe using a for loop as shown in the lesson "Code for Efficient API Extraction"

Save your notebook, commit the change to your repository and submit the repository URL for this assignment.

# Imports

In [1]:
# Standard
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# New Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

# Install tmdbsimple 
!pip install tqdm

## Login Information

In [11]:
import json
with open('/Users/jasontracey/.secret/yelp_api.json') as f: #change the path to match YOUR path!!
    login = json.load(f)

yelp_api = YelpAPI(login['API Key'], timeout_s=5.0)

# Function

In [12]:
def create_json_file(JSON_FILE,  delete_if_exists=False):

    ## Check if JSON_FILE exists    
    file_exists = os.path.isfile(JSON_FILE)
    
    ## If it DOES exist:
    if file_exists == True:
        
        ## Check if user wants to delete if exists
        if delete_if_exists==True:
            print(f"[!] {JSON_FILE} already exists. Deleting previous file...")
            ## delete file and confirm it no longer exits.
            os.remove(JSON_FILE)
            ## Recursive call to function after old file deleted
            create_json_file(JSON_FILE,delete_if_exists=False)
        else:
            print(f"[i] {JSON_FILE} already exists.")            

    ## If it does NOT exist:
    else:
        ## INFORM USER AND SAVE EMPTY LIST
        print(f"[i] {JSON_FILE} not found. Saving empty list to new file.")
        
        ## CREATE ANY NEEDED FOLDERS
        # Get the Folder Name only
        folder = os.path.dirname(JSON_FILE)
        
        ## If JSON_FILE included a folder:
        if len(folder)>0:
            # create the folder
            os.makedirs(folder,exist_ok=True)
        ## Save empty list to start the json file
        with open(JSON_FILE,'w') as f:
            json.dump([],f)  

# Set Search Parameters

In [13]:
# set our API call parameters 
LOCATION = 'San Diego, CA'
TERM = 'Sushi'

In [16]:
# Specifying JSON_FILE filename (can include a folder)
# include the search terms in the filename
JSON_FILE = "Data/results_in_progress_San_Diego_sushi.json"
JSON_FILE

'Data/results_in_progress_San_Diego_sushi.json'

In [17]:
create_json_file(JSON_FILE,  delete_if_exists=False)

[i] Data/results_in_progress_San_Diego_sushi.json not found. Saving empty list to new file.


In [18]:
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')

- 0 previous results found.


In [19]:
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
results.keys()

dict_keys(['businesses', 'total', 'region'])

In [20]:
## How many results total?
total_results = results['total']
total_results

1600

In [21]:
## How many did we get the details for?
results_per_page = len(results['businesses'])
results_per_page

20

In [22]:
# Import additional packages for controlling our loop
import time, math
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

80

In [23]:
# join new results with old list with extend and save to file
previous_results.extend(results['businesses'])  
with open(JSON_FILE,'w') as f:
     json.dump(previous_results,f)

In [24]:
from tqdm.notebook import tqdm_notebook
import time
for i in tqdm_notebook(range(n_pages)):
    # adds 200 ms pause
    time.sleep(.2) 

  0%|          | 0/80 [00:00<?, ?it/s]

In [26]:
## Create a new empty json file (exist the previous if it exists)
create_json_file(JSON_FILE, delete_if_exists=True)
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
## How many results total?
total_results = results['total']
## How many did we get the details for?
results_per_page = len(results['businesses'])
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

[!] Data/results_in_progress_San_Diego_sushi.json already exists. Deleting previous file...
[i] Data/results_in_progress_San_Diego_sushi.json not found. Saving empty list to new file.
- 0 previous results found.


80

In [29]:
for i in tqdm_notebook( range(1,n_pages+1)):
    
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    
    if (n_results + results_per_page) > 1000:
        print('Exceeded 1000 api calls. Stopping loop.')
        break
    
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
    # display(previous_results)
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    time.sleep(.1)

  0%|          | 0/80 [00:00<?, ?it/s]

ReadTimeout: HTTPSConnectionPool(host='api.yelp.com', port=443): Read timed out. (read timeout=5.0)

In [30]:
# load final results
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,3S6VtYgcXL8lRic0iV5GYQ,sushi-ota-san-diego,Sushi Ota,https://s3-media3.fl.yelpcdn.com/bphoto/rWQ8VS...,False,https://www.yelp.com/biz/sushi-ota-san-diego?a...,4386,"[{'alias': 'sushi', 'title': 'Sushi Bars'}, {'...",4.5,"{'latitude': 32.8036636, 'longitude': -117.217...",[delivery],$$$,"{'address1': '4529 Mission Bay Dr', 'address2'...",+18588808778,(858) 880-8778,11596.015168
1,O1NFJA0OI0-N77JzP7qlfQ,azuki-sushi-san-diego-2,Azuki Sushi,https://s3-media4.fl.yelpcdn.com/bphoto/OhZXVg...,False,https://www.yelp.com/biz/azuki-sushi-san-diego...,2866,"[{'alias': 'sushi', 'title': 'Sushi Bars'}, {'...",4.5,"{'latitude': 32.7296447753906, 'longitude': -1...","[delivery, restaurant_reservation, pickup]",$$,"{'address1': '2321 5th Ave', 'address2': '', '...",+16192384760,(619) 238-4760,12080.290756
2,_8-J0_Q4YmHhWTGEx1bCew,kaki-sushi-san-diego,Kaki Sushi,https://s3-media4.fl.yelpcdn.com/bphoto/8C1FWN...,False,https://www.yelp.com/biz/kaki-sushi-san-diego?...,187,"[{'alias': 'japanese', 'title': 'Japanese'}, {...",4.5,"{'latitude': 32.83017, 'longitude': -117.10431}","[delivery, pickup]",$$,"{'address1': '10428 Clairemont Mesa Blvd', 'ad...",+18585712012,(858) 571-2012,1162.353505
3,yp7SqJJuN6xedIWjz_MH1Q,sushi-diner-2-san-diego,Sushi Diner 2,https://s3-media1.fl.yelpcdn.com/bphoto/1JBW6K...,False,https://www.yelp.com/biz/sushi-diner-2-san-die...,461,"[{'alias': 'sushi', 'title': 'Sushi Bars'}]",4.0,"{'latitude': 32.791911, 'longitude': -117.100473}",[delivery],$$,"{'address1': '10330 Friars Rd', 'address2': 'S...",+16192816448,(619) 281-6448,3553.159094
4,kUv1KwH-N3ATlRfzOX7nCg,kura-revolving-sushi-bar-san-diego-2,Kura Revolving Sushi Bar,https://s3-media1.fl.yelpcdn.com/bphoto/KCWuH6...,False,https://www.yelp.com/biz/kura-revolving-sushi-...,2393,"[{'alias': 'sushi', 'title': 'Sushi Bars'}, {'...",4.0,"{'latitude': 32.8238827, 'longitude': -117.15469}",[delivery],$$,"{'address1': '4609 Convoy St', 'address2': 'St...",+18587154605,(858) 715-4605,5574.702568


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
735,QQ6n84pp7Wjpe9MVouJq1g,savoie-italian-eatery-chula-vista,Savoie Italian Eatery,https://s3-media3.fl.yelpcdn.com/bphoto/VrUrsW...,False,https://www.yelp.com/biz/savoie-italian-eatery...,899,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",3.5,"{'latitude': 32.6253363260715, 'longitude': -1...","[pickup, restaurant_reservation, delivery]",$$,"{'address1': '2015 Birch Rd', 'address2': 'Ste...",+16195915757,(619) 591-5757,25111.373025
736,VUD8MLIzHDWk4mvW5KopDg,the-poke-co-san-diego-93,The Poke Co,https://s3-media3.fl.yelpcdn.com/bphoto/q9_r4B...,False,https://www.yelp.com/biz/the-poke-co-san-diego...,45,"[{'alias': 'poke', 'title': 'Poke'}, {'alias':...",4.0,"{'latitude': 32.906063056155205, 'longitude': ...","[pickup, delivery]",$$,"{'address1': '6755 Mira Mesa Blvd', 'address2'...",+18584125311,(858) 412-5311,11690.811383
737,3e-wQsZZGW7nSA6jaCP-RQ,tuna-harbor-dockside-market-san-diego,Tuna Harbor Dockside Market,https://s3-media3.fl.yelpcdn.com/bphoto/OmTAvq...,False,https://www.yelp.com/biz/tuna-harbor-dockside-...,92,"[{'alias': 'seafoodmarkets', 'title': 'Seafood...",4.0,"{'latitude': 32.709467936905284, 'longitude': ...",[],$$,"{'address1': '598 Harbor Ln', 'address2': '', ...",+16199577121,(619) 957-7121,14593.682378
738,-4wx98G4Utc6NED2edwnvw,bowl-eater-san-diego-2,Bowl Eater,https://s3-media1.fl.yelpcdn.com/bphoto/JHjLIO...,False,https://www.yelp.com/biz/bowl-eater-san-diego-...,101,"[{'alias': 'korean', 'title': 'Korean'}, {'ali...",4.0,"{'latitude': 32.79926, 'longitude': -117.24409}","[pickup, delivery]",$$,"{'address1': '1420 Garnet Ave', 'address2': No...",+18583526620,(858) 352-6620,14186.147482
739,bmN0HYCDNa0JAvUbMTVhXg,seaview-restaurant-san-diego,Seaview Restaurant,https://s3-media3.fl.yelpcdn.com/bphoto/EKZSAw...,False,https://www.yelp.com/biz/seaview-restaurant-sa...,128,"[{'alias': 'buffets', 'title': 'Buffets'}, {'a...",4.0,"{'latitude': 32.70923, 'longitude': -117.16823}",[],$$$,"{'address1': '1 Market Pl', 'address2': '', 'a...",+16193586735,(619) 358-6735,14335.683949


In [ ]:
# check for duplicate results
final_df.duplicated(subset = 'id').sum()

In [ ]:
# save the final results to a compressed csv
final_df.to_csv('Data/final_results_San_Diego_sushi.csv.gz', compression='gzip',index=False)